In [19]:
#IMPORT DEPENDENCIES 1
import pandas
import re

from google.api_core.exceptions import InvalidArgument
import google.generativeai as palm
import config

In [22]:
#IMPORT DATA 2
#open, read and store Data to variables
#everything2.txt contains all data up to volume 32. Humanist.vol32.txt contains data of volume 32. 
with open('../assets/originalData/everything2.txt', 'r', encoding="cp437") as f:
    text = f.read()

with open('../assets/originalData/Humanist.vol32.txt', 'r', encoding="cp437") as f:
    text32 = f.read()

In [ ]:
#PREPARE DATA 3
#Concat variables
text = text + text32
#Creating list of Lines of "text"
lines = text.split("\n")

In [ ]:
#PATTERN 4
#Creating dict of diffent mail header types
patterns = {
    "1": [
        r"^\s*From: (?P<data>.*)",
        r"^\s*Subject: (?P<data>.*)",
        r"^\s*Date: (?P<data>.*)",
        r"^\s*X-Humanist: (?P<data>.*)"
    ],
    "2": [
        r"^\s*From - (?P<sender>.*)",
        r"^\s*X-Mozilla-Status: (?P<data>.*)",
        r"^\s*X-Mozilla-Status2: (?P<data>.*)",
        r"^\s*Return-path: (?P<data>.*)"
    ],
    "3": [
        r"^\s*From (?P<data>.*)",
        r"^\s*Return-Path: (?P<data>.*)",
        r"^\s*X-Original-To: (?P<data>.*)"
    ],
    "4": [
        r"^\s*Return-Path: (?P<data>.*)",
        r"^\s*X-Original-To: (?P<data>.*)",
        r"^\s*Delivered-To: (?P<data>.*)",
        r"^\s*Received: (?P<data>.*)"
    ],
    "5": [
        r"^\s*Return-Path: (?P<data>.*)",
        r"^\s*X-Spam-Checker-Version: (?P<data>.*)"
    ]
}


In [ ]:
#PATTERN KEY 5
#Function to get the key of the current pattern
def get_keys_from_value(patterns, val):
    return [k for k, v in patterns.items() if v == val]

In [ ]:
#HEADER LINES SUM 6
#Function to get the header of the current mail as str
def get_header_content_sum(start, header):
    header_len = 0
    header_content_sum = ""
    header_content_current = ""
    for i in range(start,len(lines)):
        #Mail starts after 4 Lines of Header (if header == 1)
        if (header[0] == 1):
            if (header_len == 4):
                header_len = i
                return header_content_sum, header_len
        #Mail starts with part of line "Humanist Discussion Group, Vol. "
        if ("Humanist Discussion Group, Vol. " in lines[i]):
            header_len = i
            return header_content_sum, header_len
        #Mail starts aftre "Content-Transfer-Encoding: base64"
        if (("X-PPP-Vhost: " in lines[i]) & (lines[i+1] == "") & (lines[i+4] == "") & ("Humanist Discussion Group, Vol. " not in lines[i+5]) & ("Humanist Discussion Group, Vol. " not in lines[i+6]) & ("Humanist Discussion Group, Vol. " not in lines[i+7])):
            header_len = i
            header_len = header_len + 5
            return header_content_sum, header_len
        #Mail starts after "X-PPP-Vhost: digitalhumanities.org"
        if (("X-PPP-Vhost: " in lines[i]) & (lines[i+1] == "") & ("Humanist Discussion Group, Vol. " not in lines[i+2]) & ("Humanist Discussion Group, Vol. " not in lines[i+3]) & ("Humanist Discussion Group, Vol. " not in lines[i+4]) & ("Humanist Discussion Group, Vol. " not in lines[i+5]) & ("Humanist Discussion Group, Vol. " not in lines[i+6])):
            header_len = i
            header_len = header_len + 2
            return header_content_sum, header_len
        #Mail starts after "Errors-To: humanist-bounces@lists.digitalhumanities.org"
        if (("Errors-To: " in lines[i]) & (lines[i+1] == "") & ("Humanist Discussion Group, Vol. " not in lines[i+2]) & ("Humanist Discussion Group, Vol. " not in lines[i+3]) & ("Humanist Discussion Group, Vol. " not in lines[i+4]) & ("Humanist Discussion Group, Vol. " not in lines[i+5]) & ("Humanist Discussion Group, Vol. " not in lines[i+6])):
            header_len = i
            header_len = header_len + 2
            return header_content_sum, header_len
        header_len = header_len +1
        header_content_current = lines[i]
        if (header_content_sum == ""):
            header_content_sum = header_content_current
        else:
            header_content_sum = header_content_sum + "<br>" + header_content_current
        header_content_current = ""
        


In [ ]:
#PARSE 7
#Parse through lines and adding parsed data to rows and collumns of lines_with_patters[]
lines_with_patterns = []
count = 0
x_humanist = ""
mail_content_sum = ""
mail_content_current = ""
#iterate through lines
for i in range(len(lines)):
        #iterate through pattern
        for pattern in patterns.values():
            window_length = len(pattern)
            window = lines[i: i + window_length]
            lines_patterns = zip(window, pattern)
            matches = []
            #iterate through lines_patterns
            for l, p in lines_patterns:
                m = re.match(p, l)
                if m:
                    matches.append(m.groupdict())
            #check if match fits to pattern (header type)        
            if len(matches) == len(pattern):
                #Exclusion of recognizing pattern 3 and 4 for the same mail
                if ("From " in lines[i-1]): 
                     continue
                #Get key of the current mail pattern
                keys = get_keys_from_value(patterns, pattern) 
                header = list(map(int, keys))
                count = count +1

                if header[0] == 1:
                    content_header_sum, header_len = get_header_content_sum(i, header)
                    lines_with_patterns.append( #Append a new row with current content to list
                        {
                            "mailFrom": lines[i],
                            "mailSubject": lines[i+1],
                            "mailDate": lines[i+2],
                            "mail_x-humanist": lines[i+3],
                            "mailPattern": keys,
                            "mailHeader_count": count,
                            "mailHeader_content": content_header_sum,
                            "mailContent": mail_content_sum
                        }
                    )
                    #Adding content to a specific position in the list
                    if mail_content_sum != "":
                         lines_with_patterns[count-2]["mailContent"] = mail_content_sum 
                         mail_content_sum = ""

                if (header[0] == 2) | (header[0] == 3) | (header[0] == 4) | (header[0] == 5):
                    content_header_sum, header_len = get_header_content_sum(i, header)
                    if (header_len) < len(lines):
                         x_humanist = lines[header_len]
                    lines_with_patterns.append(
                        {
                            "mail_x-humanist": x_humanist,
                            "mailPattern": keys,
                            "mailHeader_count": count,
                            "mailHeader_content": content_header_sum,
                            "mailContent": mail_content_sum
                        }
                    )
                    x_humanist = ""
                    if mail_content_sum != "":
                         lines_with_patterns[count-2]["mailContent"] = mail_content_sum
                         mail_content_sum = ""

        #Sum up the lines to capture the content of the current mail
        if header_len <= i: 
            mail_content_current = lines[i]
            if (mail_content_sum == ""):
                mail_content_sum = mail_content_current
            else:  
                mail_content_sum = mail_content_sum + "<br>" + mail_content_current
            mail_content_current = ""

    
        #Adding content to the last position in the list
        if (i+1) == len(lines):
            lines_with_patterns[count-1]["mailContent"] = mail_content_sum 
            mail_content_sum = ""
            
print(len(lines_with_patterns))

In [ ]:
#DATAFRAME 8
#Creating Dataframe
dF = pandas.DataFrame(lines_with_patterns)

In [ ]:
#DATAFRAME OPERATIONS 9
#List entries containing NaN are replaced by an empty string
dF.fillna('', inplace=True)
#revise the columns of the DataFrame
dF.loc[dF['mailDate'].isna() | (dF['mailDate'] == ''), 'mailDate'] = dF['mailContent'].str.extract(r"Date: (.+?)<br>", expand=False)
dF["mailYear"] = dF["mailDate"].str.extract(r"\b(198[7-9]|199\d|200\d|201[0-8]|8[7-9]|9[0-9])\b", expand=False)
#Specific parts of list entries are replaced by an empty string
dF["mailFrom"] = dF["mailFrom"].replace(to_replace = "From: ", value ="", regex = True) #
dF["mailSubject"] = dF["mailSubject"].replace(to_replace = "Subject: ", value ="", regex = True)
dF["mailDate"] = dF["mailDate"].replace(to_replace = "Date: ", value ="", regex = True)
dF["mailContent"] = dF["mailContent"].replace(to_replace = "<javascript:", value ="", regex = True)
dF["mail_x-humanist"] = dF["mail_x-humanist"].replace(to_replace = "Humanist Discussion Group, ", value ="", regex = True)


In [ ]:
#FORMAT YEAR 10
#Mail years are required.
#before 2000 the year is partly represented only by the last two numbers
#therefore iterate through the DataFrame and bring all years into the same format.
#If no year is specified, it is set according to the previous mail.
dF.fillna('', inplace=True)
referenceYear = "1987"
for i in range(len(dF)):
    year = dF.loc[i]["mailYear"]
    if (year == ""):
        dF.loc[i, "mailYear"] = referenceYear
    if (len(year) == 2):
        year = '19' + year
        referenceYear = year
        dF.loc[i, "mailYear"] = year
    if (len(year) == 4):
        referenceYear = year
    

In [ ]:
#TIMEZONES 11
#revise the columns of the DataFrame
dF.loc[dF["mailDate"].str.contains(r"^<br>.*", regex=True), ["mailDate"]] = ""
dF.loc[dF['mailFrom'].isna() | (dF['mailFrom'] == ''), 'mailFrom'] = dF['mailContent'].str.extract(r"<([^<>]*@[^<>]*)>", expand=False)

#operations to fill the mailTimeZone collumn with the same timezone format
dF["mailTimeZone"] = dF["mailDate"].str.extract(r"([+-]0\d{3}|[+-]1[0-8]\d{2})", expand=False)
dF.loc[dF['mailTimeZone'].isna() | (dF['mailTimeZone'] == ''), 'mailTimeZone'] = dF['mailDate'].str.extract(r"\b([A-Z]{3,4})\b", expand=False)
dF.loc[dF['mailTimeZone'].isna() | (dF['mailTimeZone'] == ''), 'mailTimeZone'] = dF['mailDate'].str.extract(r"\b([a-z]{3,4})\b", expand=False)
dF.fillna('', inplace=True)
dF.loc[dF["mailTimeZone"].str.contains(r"JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEZ", regex=True), ["mailTimeZone"]] = ""
dF.loc[dF['mailTimeZone'].isna() | (dF['mailTimeZone'] == ''), 'mailTimeZone'] = dF['mailDate'].str.extract(r"(?<=\d{2}:\d{2}\s)([A-Za-z]{2,3})", expand=False)

timezone_mapping = {
    'EST': '-0500',  # Eastern Standard Time
    'est': '-0500',  # Eastern Standard Time
    '-0500': '-0500',  # Eastern Standard Time
    'EDT': '-0400',  # Eastern Daylight Time
    'edt': '-0400',  # Eastern Daylight Time
    '-0400': '-0400',  # Eastern Daylight Time
    '+0400': '+0400', # Asia/Tehran
    'CST': '-0600',  # Central Standard Time
    'cst': '-0600',  # Central Standard Time
    '-0600': '-0600',  # Central Standard Time
    'CDT': '-0500',  # Central Daylight Time
    'cdt': '-0500',  # Central Daylight Time
    'MST': '-0700',  # Mountain Standard Time
    'mst': '-0700',  # Mountain Standard Time
    '-0700': '-0700',  # Mountain Standard Time
    '+0700': '+0700', # Asia/Bangkok
    'MDT': '-0600',  # Mountain Daylight Time
    'mdt': '-0600',  # Mountain Daylight Time
    'PST': '-0800',  # Pacific Standard Time
    'pst': '-0800',  # Pacific Standard Time
    '-0800': '-0800',  # Pacific Standard Time
    'PDT': '-0700',  # Pacific Daylight Time
    'pdt': '-0700',  # Pacific Daylight Time
    'GMT': '+0000',  # Greenwich Mean Time
    'gmt': '+0000',  # Greenwich Mean Time
    '+0000': '+0000',  # Greenwich Mean Time
    '-0000': '-0000',
    'UTC': '+0000',   # Coordinated Universal Time
    'utc': '+0000',   # Coordinated Universal Time
    'AST': '-0400',  # Atlantic Standard Time
    'ast': '-0400',  # Atlantic Standard Time
    'ADT': '-0300',  # Atlantic Daylight Time
    'adt': '-0300',  # Atlantic Daylight Time
    '-0300': '-0300',  # Atlantic Daylight Time
    'NST': '-0330',  # Newfoundland Standard Time
    'nst': '-0330',  # Newfoundland Standard Time
    '-0330': '-0330',  # Newfoundland Standard Time
    'NDT': '-0230',  # Newfoundland Daylight Time
    'ndt': '-0230',  # Newfoundland Daylight Time
    '-0230': '-0230',  # Newfoundland Daylight Time
    'AKST': '-0900',  # Alaska Standard Time
    'akst': '-0900',  # Alaska Standard Time
    '-0900': '-0900',  # Alaska Standard Time
    'AKDT': '-0800',  # Alaska Daylight Time
    'akdt': '-0800',  # Alaska Daylight Time
    'HST': '-1000',  # Hawaii Standard Time
    'hst': '-1000',  # Hawaii Standard Time
    '-1000': '-1000',  # Hawaii Standard Time
    'HDT': '-0900',  # Hawaii Daylight Time
    'hdt': '-0900',  # Hawaii Daylight Time
    'EMT': '+0100',
    'emt': '+0100',
    'BST': '+0100',  # British Summer Time
    'bst': '+0100',
    '+0100': '+0100',
    'BDT': '+0600',
    'bdt': '+0600',
    '+0600': '+0600',
    'IST': '+0530',  # Indian Standard Time
    'ist': '+0530',
    '+0530': '+0530',
    'CET': '+0100',  # Central European Time
    'cet': '+0100',
    'ITA': '+0200',
    'ita': '+0200',
    'MEZ': '+0200',
    'mez': '+0200', 
    'CEST': '+0200',  # Central European Summer Time
    'cest': '+0200',
    'EET': '+0200',  # Eastern European Time
    'eet': '+0200',
    '+0200': '+0200',
    '-0200': '-0200', #America/SaoPaulo
    'EEST': '+0300',  # Eastern European Summer Time
    'eest': '+0300',
    '+0300': '+0300',
    'JST': '+0900',  # Japan Standard Time
    'jst': '+0900',
    'KST': '+0900',  # Korea Standard Time
    'kst': '+0900',
    '+0900': '+0900',
    'AEST': '+1000',  # Australian Eastern Standard Time
    'aest': '+1000',
    '+1000': '+1000',
    'AEDT': '+1100',  # Australian Eastern Daylight Time
    'aedt': '+1100',
    '+1100': '+1100',
    'NZST': '+1200',  # New Zealand Standard Time
    'nzst': '+1200',
    '+1200': '+1200',
    'NZDT': '+1300',  # New Zealand Daylight Time
    'nzdt': '+1300',
    '+1300': '+1300',
    'MSK': '+0300',  # Moscow Time
    'msk': '+0300',
    'SGT': '+0800',  # Singapore Time
    'sgt': '+0800',
    'AWST': '+0800',  # Australian Western Standard Time
    'awst': '+0800',
    '+0800': '+0800',
    'ACST': '+0930',  # Australian Central Standard Time
    'acst': '+0930',
    '+0930': '+0930',
    'ACDT': '+1030',  # Australian Central Daylight Time
    'acdt': '+1030',
    '+1030': '+1030',
    'WAT': '+0100',  # West Africa Time
    'wat': '+0100',
    'CAT': '+0200',  # Central Africa Time
    'cat': '+0200',
    'SAST': '+0200',  # South African Standard Time
    'sast': '+0200',
    'FJT': '+1200',  # Fiji Time
    'fjt': '+1200',
    'LKT': '+0530',  # Sri Lanka Time
    'lkt': '+0530',
    'MVT': '+0500',  # Maldives Time
    'mvt': '+0500',
    '+0500': '+0500',
    'PHT': '+0800',  # Philippine Time
    'pht': '+0800',
    'IRKT': '+0800',  # Irkutsk Time
    'irkt': '+0800',
    'ULAT': '+0800',  # Ulaanbaatar Time
    'ulat': '+0800',
    'VET': '-0430',  # Venezuelan Standard Time
    'vet': '-0430',
    '-0430': '-0430',
    'CLT': '-0300',  # Chile Standard Time
    'clt': '-0300',
    'CLST': '-0300',  # Chile Summer Time
    'clst': '-0300',
    'ART': '-0300',  # Argentina Time
    'art': '-0300'
}

dF['mailTimeZone'] = dF['mailTimeZone'].map(timezone_mapping)

#fill the mailVolume collumn with the same format
dF["mailVolume"] = dF["mail_x-humanist"].str.extract(r"(Vol\. \d{1,2})", expand=False)

In [ ]:
#MULTI 12
#Search for multiple mails in mail_content and mark them with Y
dF.loc[dF["mailContent"].str.contains(r"\(1\) ---+", regex=True), ["mailMultiple"]] = "Y"
dF.loc[dF["mailContent"].str.contains(r"\(1\)---+", regex=True), ["mailMultiple"]] = "Y"
dF.loc[dF["mailContent"].str.contains(r"\[1\]---+", regex=True), ["mailMultiple"]] = "Y"

In [ ]:
#LLM 13
#using Palm API to categorize mailContent into predefined categories 

#private API Key
palm.configure(api_key=config.API_KEY)

#model = text-bison-001
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name

#iterate through DataFrame
for i in range(0, len(dF), 1):    
    prompt = "Categorize the email based on its content into the following categories. Multiple categories can be assigned to the mail: Conference Announcements, Book Announcements, Calls for Papers, Job Postings, Research Queries, Event Information, Publications, Technical Issues, Literature Review, Data Sharing, Collaboration Requests, Research Findings, Methodological Discussions, Workshop Invitations, Seminar Details, Webinar Information, Symposium Notices, Journal Articles, Online Resources, Databases. Email content: " + dF.loc[i]["mailContent"]

    #try and except to cach specific error
    try:
        completion = palm.generate_text(
            model=model,
            prompt=prompt,
            temperature=0,
            max_output_tokens=800,
        )
        #append result to mailContentCategory
        dF.loc[i, "mailContentCategory"] = completion.result
    except InvalidArgument as e:
        print(f"Fehler bei E-Mail-Index {i}: {e}. Überspringe diese E-Mail.")
        dF.loc[i, "mailContentCategory"] = "Unsupported Language"
    except Exception as e:
        print(f"Ein unerwarteter Fehler trat auf bei E-Mail-Index {i}: {e}")
        raise e

In [ ]:
#EXPORT 14
#creating second Dataframe and save Data created with palm
#dF2 = pandas.DataFrame
#dF2.to_csv('batest43.csv')

#read stored data and hand over mailContentCategory to actual DataFrame
#dF2 = pandas.read_csv('batest43.csv')
#dF['mailContentCategory'] = dF2['mailContentCategory']

#drop the collumns mailPattern and mailHeader_count wichare no longer in use
cols = ['mailPattern', 'mailHeader_count']
dF.drop(cols, inplace=True, axis = 1)

#ouput final CSV-file -> later used as import to database table
dF.to_csv('HumanistReaserchDataFinal.csv')